In [3]:
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt

class Brick(Rectangle):
    def __init__(self, xy, width, height, angle=0.0, **kwargs):
        Rectangle.__init__(self, xy, width, height, angle=0.0, **kwargs)

class Racket(Rectangle):
    def __init__(self, xy, width, height, angle=0.0, **kwargs):
        Rectangle.__init__(self, xy, width, height, angle=0.0, **kwargs)
        
    def move(self, offset):
        # Racket only moves horizontally
        self.set_x(self.get_x() + offset)

class Ball(Circle):
    def __init__(self, xy, radius, **kwargs):
        Circle.__init__(self, xy, radius, **kwargs)
        self.speed = (0,0)
        
    def hit_wall_lr(self, game):
        # check if the ball hits the left/ right wall
        if self.get_center()[0] - self.get_radius() < 0:
            return True
        elif self.get_center()[0] + self.get_radius() > game.width:
            return True
        return False
        
    def hit_wall_top(self, game):
        # check if the ball hits the top
        if self.get_center()[1] + self.get_radius() > game.height:
            return True
        return False
        
    def hit_racket(self, game):
        # check if the ball hits the racket
        if self.get_center()[1] - self.get_radius() <= game.racket_pos_y + game.racket_height:
            if (self.get_center()[0] >= game.racket.get_x() - game.racket_width/2) and self.get_center()[0] <= game.racket.get_x() + game.racket_width/2:
                return True
        return False
        
    def hit_bottom(self, game):
        # check if the ball hits the bottom
        if self.get_center()[1] - self.get_radius() <= 0:
            return True
        return False
   
    def on_speed_change(self):
        # set the center of the ball when its speed changes
        new_center = (self.get_center()[0] + self.speed[0], self.get_center()[1] + self.speed[1])
        self.set_center(new_center)
        
    def toggle_dir_y(self):
        # toggle the y direction of the ball
        self.speed = (self.speed[0], self.speed[1] * -1)
    
    def move(self, game):
        if self.speed == (0,0):
            self.speed = (-5,5)
        
        self.on_speed_change()
        
        # check if ball hits the wall or the racket
        # and act accordingly
        if self.hit_wall_lr(game):
            self.speed = (self.speed[0] * -1, self.speed[1])
            self.on_speed_change()
        elif self.hit_wall_top(game):
            self.toggle_dir_y()
            self.on_speed_change()
        elif self.hit_racket(game):
            self.toggle_dir_y()
            self.on_speed_change()
        elif self.hit_bottom(game):
            self.set_visible(False)
        
class Game():
    def animate(self,i):
        self.ball.move(self)
        self.fig.canvas.draw()
        
    def start_game(self):
        self.ball = Ball((100, 100), 10)
        self.ax.add_patch(self.ball)
        anim = FuncAnimation(self.fig, self.animate, frames=10000000000,interval=25)
        self.fig.canvas.draw()

    def racket_move(self, offset):
        offset = offset * self.move_unit
        new_x = self.racket.get_x() + offset
        
        if new_x < 0:
            new_x = 0
        elif new_x > self.width - self.racket_width:
            new_x = self.width - self.racket_width
            
        self.racket.set_x(new_x)
        self.fig.canvas.draw()
            
    def press(self, event):
        if event.key == 'enter':
            self.start_game()
        elif event.key == 'left':
            self.racket_move(-1)
        elif event.key == 'right':
            self.racket_move(1)
        elif event.key == 'up':
            self.ball.speed = (self.ball.speed[0] * 1.1, self.ball.speed[1] * 1.1)
        elif event.key == 'down':
            self.ball.speed = (self.ball.speed[0] * 0.9, self.ball.speed[1] * 0.9)

    def __init__(self):
        self.fig = plt.figure(figsize=(8, 8))
        self.ax = self.fig.add_subplot(111)
        self.ax.axis('OFF')
        
        # declare canvas size
        self.margin = 10
        self.width = 1000
        self.height = 1000
        
        # create white background with black edge
        background_rect = Rectangle((0, 0), self.width, self.height, color='white')
        background_rect.set_edgecolor('black')
        self.ax.add_patch(background_rect)
        
        # create bricks
        self.brick_width = 80
        self.brick_height = 40
        self.bricks = []
        for i in range(1,5):
            for j in range(i):
                brick = Brick(((self.width / 2 + (self.brick_width + self.margin * 3) * j), self.height / 2 + self.height / 10 * i), self.brick_width, self.brick_height, color='yellow')
                self.ax.add_patch(brick)
                self.bricks.append(brick)
        
        # create racket
        self.racket_width = self.brick_width
        self.racket_height = self.brick_height / 2
        self.move_unit = 10
        self.racket_pos_y = 20
        self.racket = Racket((self.width / 2 - self.racket_width / 2, self.racket_pos_y), self.racket_width, self.racket_height, color='blue')
        self.ax.add_patch(self.racket)
        
        self.ax.set_xlim(0,self.width + self.margin)
        self.ax.set_ylim(-self.margin,self.height)
        
        self.fig.canvas.mpl_connect('key_press_event', self.press)
        
    def draw(self):
        plt.show()
        
if __name__ == "__main__":
    game = Game()
    game.draw()



ModuleNotFoundError: No module named 'matplotlib'